### Code to time models against sample feeder images
set up code to load all images into memory  
runs all images through a model and times model at the task  
runs multiple models for comparision

In [1]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import os
import time


In [2]:
image_dir = 'home/pi/batch_test'  # relative to birdclassifier dir in jupyter nb

In [3]:
# get the classes, number space text with a line end 
classes_dict = {}
with open('NAbirdsClasses.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line:  # Ignore empty lines
            parts = line.split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                key = int(parts[0])
                value = parts[1]
                classes_dict[key] = value
# print(classes_dict)

In [4]:
def list_model_h5_files(prefix='Experiment5'):
    matching_files = []
    for filename in os.listdir():  # current dir
        if filename.startswith(prefix) and filename.endswith('.h5'):
            matching_files.append(filename)
    return matching_files

In [5]:
def load_model(model_path):
    # load the model and get input size
    model = keras.models.load_model(model_path)
    # print(model.input_shape)  
    input_shape = model.input_shape[1:3]  # height, width, channel in format (None, 224, 224, 3)
    return model, input_shape

In [6]:
def preprocess_image(image_path, target_size):
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img)
    # img_array = img_array / 255.0  # done in model? 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension, we are doing this one at a time
    return img_array

In [7]:
def batch_predict_model(image_files, image_dir, model, input_shape, classes_dict):
    # includes load time, but that needs to happen for resizing
    images_count = len(image_files)
    preprocessed_images = []
    for ii, image_file in enumerate(image_files):
        image_path = os.path.join(image_dir, image_file)
        try:
            preprocessed_images.append(preprocess_image(image_path, input_shape))
        except Exception as e:
            print(f'error processing image {image_file}: {e}')
            
    start_time = time.time()
    for ii, pimage in enumerate(preprocessed_images):
        try:
            prediction = model.predict(pimage)
            predicted_class_index = np.argmax(prediction)
            confidence = prediction[0][predicted_class_index] * 100
            # print(f'image {ii} of {images_count}: {image_file}, Prediction: {predicted_class_index}:{classes_dict[predicted_class_index]}, Confidence: {confidence:.2f}%')

        except Exception as e:
            print(f'error making prediction {image_file}: {e}')

    stop_time = time.time()
    total_run_time = stop_time - start_time
    print(f'\n run time: {total_run_time:.2f} seconds')
    return

In [8]:
model_files = list_model_h5_files(prefix='Experiment5')
model_files.append('coral.ai.mobilenet_v2_1.0_224_inat_bird_quant.tflite')  # add directories manually
model_files.append('efficientnetb0_3.h5')  # add directories manually
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

for model_filename in model_files:
    print('*************************')
    print(f'model: {model_filename}')
    try:
        model, input_shape = load_model(model_filename)
        batch_predict_model(image_files, image_dir, model, input_shape, classes_dict)
        print('')
    except Exception as e:
        print (f'error with model {model_filename} as {e}')
    

*************************
model: Experiment5:MobileNetV2MobileNetV2.h5


2025-04-22 02:38:45.872865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-22 02:38:45.904690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-22 02:38:45.904936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-22 02:38:45.905782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

error with model Experiment5:MobileNetV2MobileNetV2.h5 as weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.
*************************
model: Experiment5: EfficientNetB0: Chapter 4 all layers, one stage 50 epochs, regularization .001EfficientNetB0.h5
error with model Experiment5: EfficientNetB0: Chapter 4 all layers, one stage 50 epochs, regularization .001EfficientNetB0.h5 as No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7ff98df02c50>.
*************************
model: Experiment5:EfficientNetB0EfficientNetB0.h5


2025-04-22 02:38:51.617697: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8900
2025-04-22 02:38:51.811480: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 85.06MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1/1 [==============================] - 2s 2s/step


2025-04-22 02:38:51.821124: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 213.04MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-04-22 02:38:51.846356: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 93.35MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-04-22 02:38:51.851122: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 128.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-04-22 02:38:51.867287: W tensorflow/core/common_runtime/bfc_all

1/1 [==============================] - 0s 25ms/step

 run time: 9.92 seconds

*************************
model: Experiment5:EfficientNetB1EfficientNetB1.h5
error with model Experiment5:EfficientNetB1EfficientNetB1.h5 as No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7ff923243c40>.
*************************
model: Experiment5:MobileNetV3LargeMobileNetV3Large.h5
error with model Experiment5:MobileNetV3LargeMobileNetV3Large.h5 as weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.
*************************
model: coral.ai.mobilenet_v2_1.0_224_inat_bird_quant.tflite
error with model coral.ai.mobilenet_v2_1.0_224_inat_bird_quant.tflite as Unable to synchronously open file (file signature not found)
*************************
model: efficientnetb0_3.h5
1/1 [==============================] - 0s 25ms/step

 run time: 9.41 seconds

